# COVID19 - Brazil States

Install necessary packages for parallel computation:

```
pip install ipyparallel
ipcluster nbextension enable
pip install parallel-execute
```

To install for all users on JupyterHub, as root:
```
jupyter nbextension install --sys-prefix --py ipyparallel
jupyter nbextension enable --sys-prefix --py ipyparallel
jupyter serverextension enable --sys-prefix --py ipyparallel
```

start cluster at jupyter notebook interface

In [1]:
import urllib.request
import pandas as pd
import numpy as np

In [3]:
# Download data
import get_data
LoadData=False

if LoadData:
    get_data.get_data()

In [4]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-06,SE,TOTAL,state,8972,208,74,True,2298696.0,28.0,390.30824,0.0232
1,1,2020-06-05,SE,TOTAL,state,8577,198,73,False,2298696.0,28.0,373.12459,0.0231
2,2,2020-06-04,SE,TOTAL,state,8200,186,72,False,2298696.0,28.0,356.72399,0.0227
3,3,2020-06-03,SE,TOTAL,state,7989,180,71,False,2298696.0,28.0,347.54487,0.0225
4,4,2020-06-02,SE,TOTAL,state,7555,172,70,False,2298696.0,28.0,328.66460,0.0228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,158210,2020-03-11,BA,TOTAL,state,3,0,5,False,14873064.0,29.0,0.02017,0.0000
2245,158211,2020-03-10,BA,TOTAL,state,2,0,4,False,14873064.0,29.0,0.01345,0.0000
2246,158212,2020-03-09,BA,TOTAL,state,2,0,3,False,14873064.0,29.0,0.01345,0.0000
2247,158213,2020-03-07,BA,TOTAL,state,2,0,2,False,14873064.0,29.0,0.01345,0.0000


In [5]:
# Model

In [6]:
# states=['SP','ES','MG','RJ','PE','CE','AM']
states=['MG','ES','CE']
states

['MG', 'ES', 'CE']

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

In [7]:
# Initial parameters
paramOpt=1

if paramOpt==1:
    paramFile="data/param_optimized_DE SciPyHistMin.csv"

if paramOpt==2:
    paramFile="data/param_optimized_YaboxHistMin.csv"

if paramOpt==3:
    paramFile="data/param.csv"

dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,9.297776e+06,0,0,705,300,250,100,0.15,0.583510,0.002580
1,ES,2020-04-01,200,5.643043e+05,0,0,54,250,50,100,0.15,0.579348,0.033609
2,MG,2020-04-01,200,1.230787e+06,0,0,180,250,40,100,0.15,0.575977,0.021200
3,RJ,2020-03-20,200,8.046260e+05,0,0,888,250,50,100,0.15,0.565961,0.038091
4,CE,2020-03-20,200,9.581388e+05,0,0,804,250,50,100,0.15,0.558476,0.031493
5,PE,2020-03-20,200,7.962931e+05,0,0,846,250,100,100,0.15,0.411991,0.034598
6,AM,2020-03-20,200,7.820635e+05,0,0,858,250,100,100,0.15,0.422963,0.044164


# Initial parameter optimization

In [9]:
import ray
ncpu=len(states)
ray.init(num_cpus=ncpu)

# Load solver
import LearnerYabox as Learner #Yabox

2020-06-13 16:19:19,124	INFO resource_spec.py:212 -- Starting Ray with 155.08 GiB memory available for workers and up to 70.46 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-13 16:19:19,464	INFO services.py:1170 -- View the Ray dashboard at localhost:8267
2020-06-13 16:19:20,447	INFO resource_spec.py:212 -- Starting Ray with 154.93 GiB memory available for workers and up to 70.4 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-13 16:19:20,743	INFO services.py:1170 -- View the Ray dashboard at localhost:8267


In [10]:
allStates=False
cleanRecovered=False
version="1"

results=[]
if allStates:
    for state in states:        
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        parameters
        f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
        result = f.train.remote() 
        results.append(result)
else:
    state= "ES"
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    parameters
    cleanRecovered = False
    f=Learner.Learner.remote(state, *parameters, cleanRecovered, version)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

  1%|          | 10011/2000000 [00:57<1:21:51, 405.20it/s]


  2%|▏         | 40186/2000000 [01:48<42:18, 772.03it/s]


  5%|▍         | 90951/2000000 [02:35<20:32, 1549.55it/s]


  8%|▊         | 160461/2000000 [03:07<11:10, 2744.75it/s]


 12%|█▏        | 248865/2000000 [03:35<08:15, 3535.39it/s]


...

 59%|█████▉    | 1181953/2000000 [06:12<01:37, 8429.99it/s]


 70%|███████   | 1401975/2000000 [06:38<01:12, 8235.53it/s]


 82%|████████▏ | 1638955/2000000 [07:02<00:33, 10664.54it/s]


 95%|█████████▍| 1894431/2000000 [07:26<00:09, 11163.71it/s]


100%|█████████▉| 1999000/2000000 [07:35<00:00, 4383.94it/s] 


In [ ]:
ray.shutdown()

# Plots

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#select districts for plotting log plot
states4Plot=['MG',
               'SP',
               'ES',
               'CE',
               'RJ']

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#versio'n to identify the png file result
version = "1"

#number of cases to start plotting model in log graph - real data = 100
startCase=100

In [ ]:
#plots one district or all districts
dfparam = pd.read_csv("data/param.csv")
plotStates=True

#select states for plotting
#you need to run (solve) it before plotting
if plotStates:
    states=['SP','ES','MG','RJ','PE','CE','AM']
    allStates=True

#main state for plotting log graph or SEAIRD plot
state="ES"

#if it is not SEAIRD plot, so plot one state only
if not opt==5:
    AllStates=False

if allStates:
    for state in states:
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        ratio = query['RATIO'][0]
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        covid_plots.covid_plots(state, states4Plot, startdate,predict_range, \
                                startCase, 5, version, show=True, ratio=ratio)
else: 
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    ratio = query['RATIO'][0]
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    covid_plots.covid_plots(state, states4Plot, startdate,predict_range, \
                                startCase, opt, version, show=True, ratio=ratio)